In [326]:
%%capture
%run 01_transaction.ipynb
from IPython.display import Image

In [186]:
from random import randint as ri

Create a random tx.

In [406]:
def rt(): return TX(rh(),rh(),ri(1,99),ri(1,99)/100,0); 
print(rt())

time:	Sun Mar 28 20:37:09 2021
from:	🕣 fd040d031967c6d7...ec1
to:	🔙 b3d207ef83823e89...23f
value:	69 ether
fee:	0.79 ether
nonce:	0
hash:	👼 16f2f5d7cf8325fa...469
signed:	false


## Merkle Tree

A binary tree in which every leaf node is a hash of a specific piece of data, in our case a tx. 

In a system like Bitcoin or Ethereum the merkle root is included in the block header, not the list of txs itself. This makes it very efficient to test if a specific tx is inclued in the tree.

For more in depth [here](https://www.youtube.com/watch?v=3giNelTfeAk).

![title](https://changelly.com/blog/wp-content/uploads/2020/01/Merkle-Tree-1.png)

`MerkleTree` is a wrapper around the `mt` dictionary which stores the hashes for each height in the tree.

In [417]:
class MerkleTree:
    def __init__(self, txs):
        self.mt = {}
        leafs = [tx.hash for tx in txs]
        if len(leafs)%2!=0: leafs.append(sha('0x0'))
        self.mt['1'] = leafs
        self.merkle(leafs)
        
    def merkle(self, leafs):
        parents  = []
        while len(leafs) > 1:
            for i in range(0, len(leafs), 2):
                l = leafs[i]
                if i+1>=len(leafs): r = '0x0'
                else              : r = leafs[i+1]
                parents.append(sha(l+r))
            leafs = parents
            self.mt[f'{len(self)+1}'] = parents
            parents = []
    
    @property
    def root(self):       return self.mt[str(len(self))][0]
    def __eq__ (self, o): return self.root == o.root
    def __len__(self):    return len(self.mt)
    def __str__(self):
        s = ''
        for k,v in self.mt.items(): 
            s += f'height {k}'
            for h in v: s += f'\n {ph(h)}'
            s += '\n'
        return s

Create Merkle Tree from a list of `N` random txs.

In [418]:
N = 8
r_txs = [rt() for _ in range(N)]

mt = MerkleTree(r_txs); print(mt)

height 1
 🔪 c4db6a2933e13df2...557
 📢 7cd42a324c605612...4d5
 💏 2994369a4a890fe8...ea1
 📩 8313cd5f0a0751c5...a5a
 🕂 dc25b91a818e5ca2...898
 🔌 a6d5911ce220e56b...1d5
 💮 483b42344ad94fe7...408
 🕚 f47619ab99e6dfbc...fa7
height 2
 👾 1846fb9ba6de6190...09d
 🔧 c13d8d14b2193ae6...746
 🕅 dfeba2983e2ed82a...eba
 👶 10f4cc53c2ce97ba...c85
height 3
 📃 5dd36feb09440e78...bd9
 🔩 c3e82fbfd7226e3f...b01
height 4
 💰 4aad1b295bcad668...05f



In [368]:
assert len(mt) == 4

The merkle tree root hash is the top most node. If anything in the tree changes the root changes as well.

In [370]:
ph(mt.root)

'💽 57976e7c3ba94537...8e3'

If we change something in any tx like the `value` the whole Merkle Tree changes. Compare the hashes below with the ones above.

In [371]:
r_txs[0].value = 500

changed_mt = MerkleTree(r_txs); print(changed_mt)

height 1
 🔑 ab8be8aa0ecd93ab...dad
 💡 3bf573f4f04cc8d6...0b8
 📼 9688b8838cc9bd60...fa6
 📝 77f4691febe70b83...ea4
 💸 5271bbd33e9af25b...f8c
 📎 683a3751d9f8f3bf...e97
 💛 35cc2a1a87eedd53...12b
 💳 4d5af728ce35c10e...365
height 2
 📉 63ee2aeba7da11bc...056
 💝 3782cc1b62592cf6...f70
 💀 1a1c20815b208cf8...7dd
 🕇 e10beddef83fd5f7...1fe
height 3
 👫 05dad561c5378a0f...22b
 🔩 c39d7e1143ed5d6a...9ed
height 4
 💑 2bf8d09244a76afc...dfd



In [372]:
assert mt != changed_mt